# Simple example of time domain beamforming

Let's imagine that we have a point sound source in the far field, which is effectively creating planar waves impinging upon a two element array microphone array.

The situation is depicted in the figures below.

We begin by taking the [Jupyter Interactors](https://github.com/bokeh/bokeh/tree/2.3.0/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb) example [User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html) and modifiying it to suit our requirements.

Note that this interact does not seem to work in JupyterLab 3 at the moment.


## Import modules

In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
output_notebook()

## Set up parameters

In [ ]:
# Array characteristics
num_elements = 3
spacing = 0.1 # metres
x_location = 0.5 # metres
Arr = []
for n in range(num_elements):
    Arr.append([x_location,n*spacing])
Arr = np.array(Arr)

In [ ]:
microphone_array = [[0.5,0.05],[0.5,-0.05]]

sample_rate = 48000
sample_length = 0.001 # seconds
A = 1 # Amplitude

s_x = -0.5 # x location of source
s_y = -0.25 # y location of source
c = 340 # m/s (Speed of sound)
λ = 0.1 # metres
f = c/λ # Hz
t_0 = 0
π = np.pi

print(f'f = {f} Hz, λ = {λ} m , T = {1/f:0.2e} s') 

N = 500
x_min = -1
x_max = 1
x = np.linspace(x_min, x_max, N)
y = np.linspace(x_min, x_max, N)
t = np.arange(t_0, t_0+sample_length, 1/sample_rate)
xx, yy = np.meshgrid(x, y)
rr = np.sqrt((xx-s_x)*(xx-s_x) + (yy-s_y)*(yy-s_y))
Z = A*np.cos(2*π/λ*(rr - c*t_0))

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[Z],
        x=x_min, y=x_min,
        dw=(x_max-x_min), dh=(x_max-x_min), 
        palette="Spectral11", level="image")
p.circle(Arr[:,0],Arr[:,1],
        size=10,
        fill_color='lightgreen',
        )
p.grid.grid_line_width = 0.5

q = figure()
Y = []
col = ['red','green','blue']
for n in range(num_elements):
    r = np.sqrt((Arr[n,0]-s_x)*(Arr[n,0]-s_x) + 
                (Arr[n,1]-s_y)*(Arr[n,1]-s_y))
    F = A*np.cos(2*π/λ*(r - c*t))
    Y.append(F)
    q.line(t,F,color=col[n])

show(column(p,q), notebook_handle=True)

In [ ]:
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [ ]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

In [ ]:
show(p, notebook_handle=True)

In [ ]:
interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

In [ ]:
p.image?